# Лабораторна робота №2

**Тема:** Очищення даних та інженерія ознак

**Мета:** Підготувати набір даних підприємства до побудови моделей машинного навчання — очистити, трансформувати та закодувати дані, сформувати повний набір якісних ознак.

## Завдання 1
Здійсніть підготовку даних вашого підприємства до подальшого
моделювання використовуючи python/jupyter:
1. Замініть відсутні значення, якщо такі присутні та обґрунтуйте вибір методу(ів)
2. Проведіть кодування (Categorical Encoding) категоріальних ознак
3. Опрацюйте аномальні значення
4. Використайте підходи масштабування ознак якщо це доцільно для ваших даних

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
# Load the dataset
df = pd.read_csv("./data/ObesityDataSet_raw_and_data_sinthetic.csv")

# Check basic info
print(df.shape)
print(df.info())
df.head()

In [ ]:
# choose which columns to iterate; change to df.columns to include everything
cols = df.columns

# optionally exclude target or ID column
exclude = []  # e.g. ['ID', 'target']
cols = [c for c in cols if c not in exclude]

for col in cols:
    plt.figure(figsize=(8, 4))
    if pd.api.types.is_numeric_dtype(df[col]):
        # numeric: histogram + KDE
        sns.histplot(df[col].dropna(), kde=True, bins=30)
        plt.title(f'Histogram of {col}')
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.show()

        # optional: show a boxplot under it
        plt.figure(figsize=(8, 2))
        sns.boxplot(x=df[col].dropna())
        plt.title(f'Boxplot of {col}')
        plt.show()

    else:
        # categorical: if too many categories, show top-k
        vc = df[col].value_counts(dropna=False)
        max_categories_to_plot = 20
        if len(vc) > max_categories_to_plot:
            vc = vc.iloc[:max_categories_to_plot]
            note = f"(showing top {max_categories_to_plot} categories)"
        else:
            note = ""
        plt.figure(figsize=(8, max(2, 0.25 * len(vc))))
        sns.barplot(x=vc.values, y=vc.index, palette='viridis')
        plt.title(f'Count plot of {col} {note}')
        plt.xlabel('Count')
        plt.ylabel(col)
        plt.show()

Як показано на графіках, розподіл атрибутів дуже розсіяний, і будь-який із атрибутів важко назвати *рівномірно* розподіленим. Відповідно, якщо в датасеті будуть траплятися пропущені комірки, заповнювати їх ми будемо значенням *медіани*, оскільки середнє значення атрибутів помітно зміщене.

Стосовно категоріальних даних, пропущені комірки (**до** їхнього кодування) будемо замінювати значенням *моди*, тобто найбільш поширеним значенням у вибірці. Як ми побачимо за результатом коду нижче, потреби створювати нову категорію для заміни відсутніх даних не буде (а отже ми в безпеці від перенавчання моделі).

In [ ]:
# Looking for empty values
print(df.isnull().sum())

# Filling empty values if present
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':
            # if data type is object, fill with mode
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            # if data type is numeric, fill with median
            df[col].fillna(df[col].median(), inplace=True)

df.head()

In [ ]:
# Turning categorical data into numeric using Label Encoding
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df.head()

In [ ]:
minmax_scaler = MinMaxScaler()

scaled_columns = df.select_dtypes(include=['int64', 'float64']).columns
df[scaled_columns] = minmax_scaler.fit_transform(df[scaled_columns])

df.head()

**Масштабування ознак**

Всі числові ознаки нормалізуються за допомогою MinMaxScaler. Таким чином ми обмежили значення атрибутів до проміжку `[0, 1]`, хоч і даний метод досить чутливий до викидів. На щастя, у нашому датасеті такі відсутні, отже ми можемо використовувати його без остраху — інакше ми користувалися б StandardScaler.

## Завдання 2
Сформуйте "Звіт про очищення даних"

### Звіт про очищення даних

**1. Відсутні значення:**
- Пропуски в даних відсутні
- На випадок появи пропусків (при додаванні нових даних) введено застовання наступних кроків:
    - Пропуски у числових ознаках замінено на медіану.
    - Пропуски у категоріальних ознаках замінено на найчастіше значення (моду).

**2. Кодування категоріальних ознак:**
- Всі категоріальні ознаки закодовано числовими значеннями за допомогою LabelEncoder.

**3. Обробка аномальних значень:**
- Поверхневий огляд даних показує відсутність значущих аномалій. Задля уникнення випадкового видалення значущої інформації, цей крок був пропущений.

**4. Масштабування ознак:**
- Всі числові ознаки стандартизовано за допомогою MinMaxScaler.

**Результат:**
Дані готові до побудови моделей машинного навчання: відсутні пропуски, категорії закодовані, ознаки масштабовані.

## Завдання 3
Обґрунтуйте методи інженерії даних, які були використані для підготовки даних вашого підприємства.

### Обґрунтування методів інженерії даних

**1. Обробка пропусків:** Пропуски у даних можуть призвести до некоректної роботи моделей та втрати інформації. Для числових ознак обрана медіана, оскільки вона стійка до викидів. Для категоріальних — мода, що дозволяє зберегти найбільш поширені значення.

**2. Кодування категоріальних ознак:** Більшість алгоритмів машинного навчання працюють лише з числовими даними. LabelEncoder дозволяє швидко перетворити категорії у числа, що спрощує подальше моделювання.

**3. Масштабування ознак:** Стандартизація ознак забезпечує коректну роботу алгоритмів, чутливих до масштабу (наприклад, SVM, нейронні мережі). Це особливо важливо для задач класифікації ризиків, де ознаки мають різні одиниці виміру.

**Висновок:** Обрані методи дозволяють підготувати якісний набір даних для побудови моделі прогнозування ризику ожиріння, що допоможе HealthGuard Insurance персоналізувати страхові тарифи та зменшити витрати на покриття хронічних захворювань.

In [ ]:
import sqlite3

df.to_csv("./data/obesity_data_processed.csv", index=False)

start_date = pd.to_datetime('2025-01-01')
end_date = pd.to_datetime('2025-12-31')

n = len(df)
random_dates = pd.to_datetime(
    np.random.randint(start_date.value // 10**9, end_date.value // 10**9, n), unit='s'
)
df['timestamp'] = random_dates

conn = sqlite3.connect('obesity_data_processed.db')
df.to_sql("obesity_data_processed", conn, if_exists='replace', index=False)
conn.close()